In [1]:
using DataFrames
using JuMP, Gurobi
using Gadfly

    .+(AbstractArray{S<:Union(AbstractJuMPScalar,GenericQuadExpr{Float64,Variable},GenericAffExpr{CoefType,VarType},GenericNormExpr{2,Float64,Variable},GenericNorm{P,Float64,Variable}),N},AbstractArray{T<:Number,N}) at C:\Users\huntington\.julia\v0.3\JuMP\src\operators.jl:565
is ambiguous with: 
    .+(Union(DataArray{T,N},PooledDataArray{T,R<:Integer,N}),AbstractArray{T,N}...) at C:\Users\huntington\.julia\v0.3\DataArrays\src\broadcast.jl:284.
To fix, define 
    .+(Union(DataArray{S<:Union(AbstractJuMPScalar,GenericQuadExpr{Float64,Variable},GenericAffExpr{CoefType,VarType},GenericNormExpr{2,Float64,Variable},GenericNorm{P,Float64,Variable}),N},PooledDataArray{S<:Union(AbstractJuMPScalar,GenericQuadExpr{Float64,Variable},GenericAffExpr{CoefType,VarType},GenericNormExpr{2,Float64,Variable},GenericNorm{P,Float64,Variable}),R<:Integer,N}),AbstractArray{T<:Number,N})
before the new definition.
    .+(AbstractArray{S<:Union(AbstractJuMPScalar,GenericQuadExpr{Float64,Variable},GenericAffEx

In [ ]:
m = Model(solver=GurobiSolver())


iGenLoad = readtable("BatteryModel_GenLoad.csv",header=true)
iPrices = readtable("BatteryModel_Prices.csv",header=true)
#iParameters = readtable("BatteryModel_Parameters.csv",header=true)
#iScheduledLoadsData = readtable("SchedulableLoads_Parameters.csv", header=true) 
iThermalParameters = readtable("ComplexThermal_Coeff.csv", header=true) 
iThermalFactors = readtable("ComplexThermal_Factors.csv", header=true) 

T = length(iPrices[:Hour]) 
pHours = iPrices[:Hour][1:T]

pBuyEnergy = iPrices[:pBuy_Energy][1:T]
pSellEnergy = iPrices[:pSell_Energy][1:T]
pNonControllableLoad = iGenLoad[:pNonControllableLoad][1:T]


n=2 #thermal model order
k=6 #number of exogenous variables

In [ ]:
##Noras Thermal Model:

cCV= [((n+1)*k)+n] #constant coefficients - load from first tab in nora's spreadsheet
cBTM = [T+n, k+1] #weather and build input data


# determine temp for t = 1 to n, from lag variables
for t=1:n 
    @addConstraint(m, vTempInt[t] == 
        cCV[1]*cBTM[1,t+1]+cCV[2]*cBTM[1,t]                         #past zonal temps
    +cCV[3]*cBTM[2,t+2]+cCV[4]*cBTM[2,t+1]+cCV[5]*cBTM[2,t]     #current and past ambient temps
    #UPDATE column numbers
    +cCV[3]*cBTM[2,t+2]+cCV[4]*cBTM[2,t+1]+cCV[5]*cBTM[2,t]     #current and past Q radiative
    +cCV[3]*cBTM[2,t+2]+cCV[4]*cBTM[2,t+1]+cCV[5]*cBTM[2,t]     #current and past Q convective
    +cCV[3]*cBTM[2,t+2]+cCV[4]*cBTM[2,t+1]+cCV[5]*cBTM[2,t]     #current and past Q window solar
    +cCV[3]*cBTM[2,t+2]+cCV[4]*cBTM[2,t+1]+cCV[5]*cBTM[2,t]     #current and past Q surface solar
    +cCV[3]*cBTM[2,t+2]+cCV[4]*cBTM[2,t+1]+cCV[5]*cBTM[2,t])     #current and past ground temps
end


# determine temp for t > 1 from existing temp and lag variables
for t = (n+1):T
   
    @addConstraint(m, vTempInt[t] == 
    cCV[1]*vTempInt[t-1]+cCV[2]*vTempInt[1,t-2]                         #past zonal temps
    +cCV[3]*cBTM[2,t]+cCV[4]*cBTM[2,t-1]+cCV[5]*cBTM[2,t-2]     #current and past ambient temps
    #UPDATE column numbers
    +cCV[3]*cBTM[2,t]+cCV[4]*cBTM[2,t-1]+cCV[5]*cBTM[2,t-2]     #current and past Q radiative
    +cCV[3]*(cBTM[2,t]+Q_cool[t])+cCV[4]*(cBTM[2,t-1]+Q_cool[t-1])+cCV[5]*(cBTM[2,t-2]+Q_cool[t-2])     #current and past Q convective
    +cCV[3]*cBTM[2,t]+cCV[4]*cBTM[2,t-1]+cCV[5]*cBTM[2,t-2]     #current and past Q window solar
    +cCV[3]*cBTM[2,t]+cCV[4]*cBTM[2,t-1]+cCV[5]*cBTM[2,t-2]     #current and past Q surface solar
    +cCV[3]*cBTM[2,t]+cCV[4]*cBTM[2,t-1]+cCV[5]*cBTM[2,t-2])     #current and past ground temps
    
    ##
    Q_cool[t]==vP_hvac[t]*COP  ##all Qs in watts, all temps in deg C
end

